# CRUD

CRUD (Create, Read, Update, Delete) is an acronym for ways one can operate on stored data.

## Data Model

Some advantages of the MongoDB
* It don't require the documents (rows in the SQL) have the same schema 
  * Thats mean the `fields` don't need to be in all documents for a given collection. 
  * Also, the `values` don't need to be the same type for all.
* In practice, the `documents` in a collection share a similar structure

## Documents Structure

### Embedded Data (denormalized)

```js
{ _id: <ObjectId1>,
username:"123xyz",
contact:{
    phone:"+51970854623",
    email:"soul@soulmail.com"
    },
acces: {
    level:5,
    group:"dev"
    },
workingDays: [
    "Monday", "Tuesday", "Wednesday","Friday"
]

}
```

Above technically, the field `contact` should be in other collection like `user-contact`, but it is embedded.

The advantage of this structure model is a single document can retrive all the data related about a user, or sales, or any entity.

This posibility the property of `atomicity`, _all-or-nothing_. In each transaction, if we do some changes, thi change for the user and the related data. This would not happen in the normalized data.

> A transaction is a sequence of operations performed on a database as a single logical unit of work.

### References (normalized)

<span style = "text-decoration:underline"> User <span>

```js
{ _id: <ObjectId1>,
username:"123xyz"
}
```

<span style = "text-decoration:underline"> User-access <span>

```js
{
    _id: <ObjectId4>,
    user_id: <ObjectId1>,
    level:5,
    group:"dev"
}
```

<span style = "text-decoration:underline"> User-Contact <span>

```js
{
    _id: <ObjectId3>,
    user_id: <ObjectId1>,
    phone:"+51970854623",
    email:"soul@soulmail.com"
    }
```
<span style = "text-decoration:underline"> User-WorkingDays  <span>
```js
{
    _id: <ObjectId2>,
    user_id: <ObjectId1>,
    ["Monday", "Tuesday", "Wednesday","Friday"
]
}

```

All related data are normalized across multiple documents


### Models with `Pydantic`

The models are classes that inherit from `BaseModel`

The models are requirements from the some `endpoint` in an API.

> Pydantic guarantees the types and the constraints of the output model, not the input data.

For instance, if we set that a given field, e.g. `age`, and this must be `integer`, the representation of that would be `age:int`. If we passing like values `'15'` (string type) instead a number, then pydantic convert this value into int to  `guarantees` the types of output model.

When this the convertion is not success raise error, e.g., if we pass `"sun"`, this can't convert to `int`

In [85]:
from pydantic import BaseModel, ValidationError, validator,constr
from datetime import datetime
from typing import List, Optional, Any, Dict, Float
from sqlalchemy import Column, Integer, String

In [121]:
# Model for Product
class Products(BaseModel):
    product_id:int
    title_dom:str
    brand_dom:str
    original_price_dom:float
    discount_dom:float=0
    main_price_dom:float
    lowest_price_dom:float=None
    description_dom:str=''

Here `Product` is a model that contain 8 fiels, `product_id` which is an integer and is required since don't have a default value like the field `discount_dom`, and aso we can describe the other fields.

In [119]:
products = Products(
    product_id='1001',
    title_dom="ADILETTE AQUA SLIDES",
    # brand_dom="Adidas Company",
    brand_dom=80,
    original_price_dom=27,
    discount_dom=0,
    main_price_dom=19,
    description_dom=80
)


`products` is an instance of the model `Product`. We can use the method `.dict()` to see the values of the model.

Also the values of the instance can be changed

In [41]:
products.dict()

{'product_id': 1001,
 'title_dom': 'ADILETTE AQUA SLIDES',
 'brand_dom': 'Adidas Company',
 'original_price_dom': 27.0,
 'discount_dom': 0.3,
 'main_price_dom': 18.9,
 'lowest_price_dom': None,
 'description': ''}

In [67]:
class Stars(BaseModel):
    star_1:int=0
    star_2:int=0
    star_3:int=0
    star_4:int=0
    star_5:int=0

In [68]:
# Sales
class Sales(BaseModel):
    # The `id` of the transaction 
    # is generated by a generator
    items_dom : List[Products]
    date_dom:datetime
    seller_id:int
    stars: Stars

We can use the model inside other model, like above

In [71]:
sales = Sales(
    items_dom=[Products(
        product_id=1001,
        title_dom="ADILETTE AQUA SLIDES",
        brand_dom="Adidas Company",
        original_price_dom=27,
        discount_dom=0,
        main_price_dom=0.7*27
    )],
    date_dom=datetime(2020, 8, 19),
    seller_id=2020,
    stars=Stars()
)


In [72]:
sales.dict()

{'items_dom': [{'product_id': 1001,
   'title_dom': 'ADILETTE AQUA SLIDES',
   'brand_dom': 'Adidas Company',
   'original_price_dom': 27.0,
   'discount_dom': 0.0,
   'main_price_dom': 18.9,
   'lowest_price_dom': None,
   'description_dom': ''}],
 'date_dom': datetime.datetime(2020, 8, 19, 0, 0),
 'seller_id': 2020,
 'stars': {'star_1': 0, 'star_2': 0, 'star_3': 0, 'star_4': 0, 'star_5': 0}}

#### ORM Mode (aka Arbitrary Class Instances)

1. The Config property `orm_mode` must be set to True.
2. The special constructor `from_orm` must be used to create the model instance.

More information here
https://pydantic-docs.helpmanual.io/usage/models/#orm-mode-aka-arbitrary-class-instances

### Handly Error and Custom

Here appear the concepts about the `validator`


In [3]:
from pydantic import BaseModel, ValidationError, validator, PydanticValueError

In [27]:
class DoNotExistCompany(PydanticValueError):
    code = "dont_belong_to_company"
    msg_template= 'the company should belong to our database, got "{wrong_value}"'

class Sellers(BaseModel):
    seller_id:int
    seller_name_dom:str
    
    @validator('seller_name_dom')
    def value_must_by_str(cls, v):
        v = v.title()
        if v not in ['Inversiones Fabris S.A.', 'Negocios La Leña S.A.C']:
            raise DoNotExistCompany(wrong_value=v)
        return v

Here we validate if the the name of the seller is in a list. For a better validation, we can pass the vlaidation with the our database of the our sellers

In [28]:
seller = Sellers(
    seller_id=40,
    seller_name_dom='Negocios la leña s.a.c'
)

### Helper Functions

* `parse_obj`: pass a `dict`.
* `parse_raw`: `str` or `bytes` that the shape of a dict.
* `parse_file`: take the path of the file.

In [32]:
seller = Sellers.parse_obj(
    {'seller_id': 40, 
    'seller_name_dom': 'Negocios la leña s.a.c'}
)

In [36]:
seller = Sellers.parse_raw('{"seller_id": 40, "seller_name_dom": "Negocios la leña s.a.c"}'
                           )

In [40]:
from codecs import unicode_escape_encode
from pathlib import Path

path = Path('data.json')
path.write_text('{"seller_id": 40, "seller_name_dom": "Negocios la leña s.a.c"}',encoding='utf-8')
m = Sellers.parse_file(path)
print(m)

seller_id=40 seller_name_dom='Negocios La Leña S.A.C'
